# a: Mathematical intermezzo

Basis of vector $\vec{v}_i$:

$$\vec{v}_i = \begin{bmatrix} v_{i1} \\ \vdots \\ \vdots \\ v_{in} \end{bmatrix}$$

Where the basis is orthogonal, i.e.:

$$\vec{v}_j^T\vec{v}_i = \delta_{ij}$$

SHow that an orthogonal or unitary transformation

$$\vec{w}_i = U\vec{v}_i$$

dot product $w_i^T w_i$:

$$w_i^T w_i = Uv_i^T Uv_i$$

In [1]:
#using Plots
using LinearAlgebra
using Test

In [2]:
function eigenvalue_test(N::Int64)
    # setting up and diagonalizing a matrix
    
    h = 1.0/N    # step size
    a = -1.0/h^2 # sub- and superdiagonal elements
    d = 2.0/h^2  # diagonal elements


    A = Matrix{Float64}(undef, N, N)
    for i = 1:N
        for j = 1:N
            if (j == i-1) || (j == i+1)
                A[i, j] = a
            elseif (j == i)
                A[i, j] = d
            else
                A[i, j] = 0
            end
        end
    end
    
    A = Symmetric(A)
    
    lambda_num = eigvals(A) # numerical eigenvales
    lambda_an = [d + 2*a*cos(j*pi/(N+1)) for j = 1:N] # analytical eigenvalues
    
    epsilon = 1e-10 # 
    @testset "Eigenvalues" begin
        for i = 1:N @test abs(lambda_an[i] - lambda_num[i]) < epsilon end
    end
    
    println("Test with N = $(N) successful!")
end

eigenvalue_test(10)

Test Summary: | Pass  Total
Eigenvalues   |   10     10
Test with N = 10 successful!


In [3]:
""" Implementing the Jacobi Method """

function find_max(A)
    """Find the maximum value of the non-diagonal elements"""
    
    N = size(A)[1]
   
    max = 0
    k = 0 
    l = 0
    for i = 1:N
        for j = 1:N
            if j != i && abs(A[i, j]) > abs(max)
                max = A[i, j]
                k = i
                l = j
            end
        end
    end
    
    return max, k, l
    
end


function create_matrix(N::Int64, quantum=false)
    """Function for setting up a tridiagonal, symmetric matrix"""
    
    h = 1.0/N    # step size
    a = -1.0/h^2 # sub- and superdiagonal elements
    d = 2.0/h^2  # diagonal elements
    
    # set up and initialize the matrix    
    A = zeros(Float64, N, N)
    
    A[1, 1] = d
    A[1, 2] = a
    
    for i = 2:N-1
        A[i, i-1] = a
        A[i, i+1] = a
        A[i, i] = d
    end
    
    A[N, N] = d
    A[N, N-1] = a
    
    return A
    
end

function jacobi(A::Matrix)
    
    N = size(A)[1] # extract the dimension of A

    # matrix for storing the eigenvectors of A
    R = Matrix{Float64}(I, N, N)   
    
    epsilon = 1e-8 # convergence threshold
    off_A = find_max(A)[1] # initial maximum value of non-diagonal elements
    
    max_iters = N*N*N
    counter = 0
    
    """Use the built-in functions for calculating the eigenvalues and vectors of A, and time it"""
    julia_eig = @timed eigen(A)
    t_linalg = julia_eig[2]
    eigvalss = julia_eig[1].values
    
    """Perform the Jacobi method and time it"""

    t_jacobi = @elapsed while abs(off_A) > epsilon && counter < max_iters

        # find maximum of non-diagonal elements and their indices
        max, k, l = find_max(A) 
        

        a_kk = A[k, k]
        a_ll = A[l, l]

        # calculate the values of cos, sin, tan
        tau = (a_ll - a_kk)/(2*A[k, l])

        if tau > 0 
            t = 1.0/(tau + sqrt(1 + tau*tau))
        else
            t = -1.0/(-tau + sqrt(1 + tau*tau))
        end

        c = 1.0/(sqrt(1 + t*t))
        s = t*c


        for i = 1:N
            if i != k && i != l
                a_ik = A[i, k]
                a_il = A[i, l]
                A[i, k] = c*a_ik - s*a_il
                A[k, i] = A[i,k]
                A[i, l] = c*a_il + s*a_ik
                A[l, i] = A[i, l]
            end

            # iterate the eigenvectors
            r_ik = R[i, k]
            r_il = R[i, l]
            R[i, k] = c*r_ik - s*r_il
            R[i, l] = c*r_il + s*r_ik
        end


        # calulate the new diagonals, and hard-code the relevant off-diags to zero
        A[k, k] = c*c*a_kk - 2.0*c*s*A[k, l] + s*s*a_ll
        A[l, l] = s*s*a_kk + 2.0*c*s*A[k, l] + c*c*a_ll
        A[k, l] = 0.0
        A[l, k] = 0.0

        off_A = find_max(A)[1]
        counter += 1

        # test orthogonality of new eigenvectors every 100 iterations
        if (counter % 100 == 0)
            for i = 1:N
                for j = 1:N
                    if i != j @test dot(R[:,i], R[:,j]) < epsilon end
                end
            end
        end # end if

    end # end of while
    
    # return a tuple with the eigenvalyes, along with no. of transformations, and times
    return (sort(diag(A)), eigvalss), counter, t_jacobi, t_linalg
        
end


jacobi (generic function with 1 method)

In [4]:

function analyse_jacobi()
    dims = [4, 10, 50, 100]      # matrix dimensions to test
    num_trans = zeros(Int64, length(dims)) # array for storing number of transformations
    t = zeros(2, length(dims))             # array for storing the benchmark timed
    error = zeros(Float64, length(dims))   # array for storing the error between the analytical and computed eigvals
    
    lambda_comp = 0
    num = 0
    
    
    for (i, d) in enumerate(dims)
        A = create_matrix(d)
        lambda_an = [A[1] + 2*A[2]*cos(j*pi/(d+1)) for j = 1:d]  # analytical eigenvalues

        # take an average CPU time of 10 calculations
        for k = 1:10
            results = jacobi(A)
            num, t_jacobi, t_linalg = results[2:end]
            
            t[1, i] += t_jacobi
            t[2, i] += t_linalg
            
            # store the eigvals and number of transformations
            if k == 10
                lambda_comp = results[1][1]
            elseif k == 1 # for some reason the num variable is zero for all k != 1..
                num_trans[i] = num
            end
        end
        
        # calculate the maximum relative error
        error[i] = maximum(map(log10, map(abs, (lambda_comp .- lambda_an) ./ lambda_an)))
        
        t[1, i] /= 10
        t[2, i] /= 10
        
    end
    
    for i = 1:length(dims)
        println("n = $(dims[i])")
        println("# transformations: $(num_trans[i])")
        println("log10 max error: $(error[i])")
        println("Jacobi Algorithm time: $(t[1, i])")
        println("Eigen time: $(t[2,i])")
        println("Jacobi time / Eigen time: $(t[1, i]/t[2, i])")
        println()
    end
end

analyse_jacobi()

n = 4
# transformations: 6
log10 max error: -15.132851170897386
Jacobi Algorithm time: 6.501e-7
Eigen time: 0.028745370599999993
Jacobi time / Eigen time: 2.2615815570664452e-5

n = 10
# transformations: 153
log10 max error: -14.755940159070587
Jacobi Algorithm time: 2.332e-5
Eigen time: 0.0007705601000000001
Jacobi time / Eigen time: 0.030263700391442532

n = 50
# transformations: 4316
log10 max error: -14.015893961294477
Jacobi Algorithm time: 0.022540750100000007
Eigen time: 0.0084774601
Jacobi time / Eigen time: 2.6589037086709504

n = 100
# transformations: 17663
log10 max error: -12.922510781029182
Jacobi Algorithm time: 0.3564059401
Eigen time: 0.0201363497
Jacobi time / Eigen time: 17.69963004267849



# c: implementing tests in the code

In [7]:
function test_max_value()
    """
    A unit test for the max-value function. 
    Tests whether the find_max-function finds the correct off-diagonal maximum value.
    """
    
    # set up a 5x5 matrix with know values
    A = Matrix{Float64}(undef, 5, 5)
    for i = 1:5
        for j = 1:5
            if (i != j)
                A[i, j] = i*j
            else 
                A[i, j] = 0
            end
        end
    end
    
    max_known = 4*5
    max_func = find_max(A)[1] 
    epsilon = 1e-14 
    
    @test abs(max_known - max_func) < epsilon
    #println("Find max value test passed!")
end

test_max_value()

Test Passed

# e: extending our machinery to quantum mechanics

To extend our machinery to quantum mechanics, we simply need to construct a different matrix to diagonalize

In [125]:
function create_quantum_matrix(N::Int64, rho_max::Float64)
    """Create a matrix for the specialized quantum mechanics case"""
    
    h = rho_max/(N+1) # step size
    d = 2.0/(h*h)     # diagonal elements (without the added potential)
    a = -1.0/(h*h)    # super- and subdiagonal elements
    
    # initialize rho and the potential
    rho = [i*h for i = 1:N]
    V = [r*r for r in rho]
    
    # initialize the matrix
    A = zeros(Float64, N, N)
    
    A[1, 1] = d + V[1]
    A[1, 2] = a
    
    for i = 2:N-1
        A[i, i-1] = a
        A[i, i+1] = a
        A[i, i] = d + V[i]
    end
    
    A[N, N] = d + V[N]
    A[N, N-1] = a
    return A
    
end


create_quantum_matrix (generic function with 1 method)

In [ ]:
# analysing the quantum case

dims = [4, 10, 50, 100, 200]
rho_max = [3, 5, 8, 10, 15, 20, 25]

error = zeros(Float64, length(dims), length(rho_max))
times_jacobi = copy(error)
times_julia = copy(error)
num_trans = copy(error)

eig_exact = [3, 7, 11, 15]
for (i, d) in enumerate(dims)

    for j in 1:length(rho_max)
        Q = create_quantum_matrix(d, rho_max[j])
        eigs, num_trans[i,j], times_jacobi[i, j], times_julia[i, j] = jacobi(Q)
        
        eigs = eigs[1][1:4]
        error[i, j] = sum(map(abs, (eigs .- eig_exact)))
    end
end



In [34]:
function bisect(c::Array, b::Array)
    """A function for calculating the eigen values of a symmetrical,
    tridiagonal matrix by method of bisection.
    
    c : array for diagonal elements
    b : array for subdiagonal elements
    """
    
    n = length(c) # order of the tridiagonal matrix
    
    beta = [b[i]*b[i] for i = 1:n] # array for storing the squares of the sub-diagonal elements
    
    x = zeros(Float64, n) # array for storing the computed eigenvalues
    
    eps1 = 1e-8      # precision of computed eigenvalues
    eps2 = 0         # accuracy of the results
    relhef = 1e-17   # machine precision
    
    
    """begin procedure"""
    
    # calculation of xmin, xmax
    xmin = c[n] - abs(b[n])
    xmax = c[n] + abs(b[n])
    
    for i = n-1:-1:1
        h = abs(b[i] + abs(b[i+1]))
        if (c[i] + h > xmax) xmax = c[i] + h end
        if (c[i] - h < xmin) xmin = c[i] - h end
    end # end i
    
    eps2 = relhef*(if xmin + xmax > 0 xmax else xmin end)
    if eps1 <= 0 eps1 = eps2 end
    eps2 = 0.5*eps1 + 7*eps2
    
    # inner block
    a = 0; k = 0
    q = 0.0; x1 = 0.0; xu = 0.0; x0 = 0.0
    wu = copy(x)
    
    x0 = xmax
    for i = 1:n
        x[i] = xmax
        wu[i] = xmin
    end # end i
    z = 0
    
    # loop for the k-th eigenvalue
    
    for k = n:-1:1
        xu = xmin
        for i = k:-1:1
            if (xu < wu[i]) xu = wu[i] end
        end # end i
        
        if (x0 > x[k]) x0 = x[k] end
        x1 = (xu + x0)/2
        while (x0 - xu) > (2*relhef*(abs(xu) + abs(x0)) + eps1) #&& z < n*n
            x1 = (xu + x0)/2
            z += 1
            
            a = 0; q = 1
            for i = 1:n
                q = c[i] - x1 - (if q != 0 beta[i]/q else abs(b[i])/relhef end)
                if q < 0 a += 1 end
                
                end # end i
            
            if a < k
                if a < 1
                    wu[1] = x1
                    xu = wu[1]
                else
                    wu[a+1] = x1
                    xu = wu[a+1]
                    if x[a] > x1 x[a] = x1 end
                end
            else
                x0  = x1
            end
         
        end # end while
        x[k] = (x0 + xu)/2
    end # end k
    
    return x
    
end # end of function

    
    


a = [200 for i=1:10]
b = [-100 for i=1:10]
b[1] = 0

bisect(a, b)

10-element Array{Float64,1}:
 100.00000000291038
 100.00000000291038
 100.00000000291038
 116.91699740185868
 171.5370323479874 
 228.4629676520126 
 283.0830025981413 
 299.9999999970896 
 299.9999999970896 
 299.9999999970896 

In [33]:
jacobi(create_matrix(10))[1][1]

10-element Array{Float64,1}:
   8.101405277100511
  31.749293433763768
  69.02785321094292 
 116.91699739962272 
 171.53703234534268 
 228.462967654657   
 283.08300260037737 
 330.972146789057   
 368.25070656623626 
 391.89859472289965 

In [30]:
# comparing Julia's eigenvalue solver for a normal, sparse matrix and for a symmetrical tridiagonal solverT

A = create_matrix(10000)
@time eigvals(A);

d = A[1]
e = A[2]

dv = [d for i = 1:size(A)[1]]
ev = [e for i = 1:size(A)[1]-1]

A_tri = SymTridiagonal(dv, ev)

@time eigvals(A_tri);

 85.870299 seconds (19 allocations: 766.526 MiB, 0.07% gc time)
  1.589166 seconds (12 allocations: 313.000 KiB)


# Lanczos' algorithm



In [26]:
function lanczos(A::Array)
    """A function for converting the input matrix A to a 
    tridiagonal matrix using the Lanczos algorithm"""
    
    n = size(A)[1] # input matrix order
    
    """initial step"""
    # set up an arbitrary vector with norm 1
    v_old = zeros(Float64, n)
    v_old[1] = 1
    
    w_old_mark = A*v_old
    alpha_old = w_old_mark'*v_old
    w_old = w_old_mark .- alpha_old*v_old
    
    alpha = zeros(Float64, n) # diagonal elements of resulting tridiagonal matrix
    beta = zeros(Float64, n-1)        # sub and superdiagonal ----::-----
    
    alpha[1] = alpha_old
    
    """ remaining iterations """
    for j = 2:n
        beta[j-1] = norm(w_old)
        
        if beta[j-1] > 1e-14
            v_new = w_old ./ beta[j-1]
        else
            v_new = zeros(Float64, n)
            v_new[j] = 1
        end
        
        w_new_mark = A*v_new
        alpha[j] = w_new_mark'*v_new
        w_new = w_new_mark - alpha[j]*v_new - beta[j-1] .* v_old
    end
    
    # construct the matrix as a symmetrical tridiagonal matrix
    T = SymTridiagonal(alpha, beta)
    
    return T
end

lanczos (generic function with 2 methods)

In [35]:
A = create_matrix(1000)
T = @time lanczos(A)

@time eigvals(T);

  0.702081 seconds (6.00 k allocations: 46.509 MiB, 1.35% gc time)
  0.018172 seconds (9 allocations: 31.891 KiB)
